In [44]:
import requests
import feedparser

# Base api query url
base_url = "http://export.arxiv.org/api/query?"

# Search parameters
query_params = {
    'search_query': "all:rag",
    'start': 0,  # retreive the first 5 results
    'max_results': 5,
    'sortBy': "submittedDate",
    'sortOrder': "descending"
}
query = urllib.parse.urlencode(query_params)

# perform a GET request using the base_url and query
response = requests.get(base_url + query)

# parse the response using feedparser
feed = feedparser.parse(response.text)

# print out feed information
print("Feed title: %s" % feed.feed.title)
print("Feed last updated: %s" % feed.feed.updated)

# print opensearch metadata
print("totalResults for this query: %s" % feed.feed.opensearch_totalresults)
print("itemsPerPage for this query: %s" % feed.feed.opensearch_itemsperpage)
print("startIndex for this query: %s" % feed.feed.opensearch_startindex)

# Run through each entry, and print out information
for entry in feed.entries:
    print("e-print metadata")
    print("arxiv-id: %s" % entry.id.split("/abs/")[-1])
    print("Published: %s" % entry.published)
    print("Title:  %s" % entry.title)

    # Authors
    authors = entry.authors
    if authors:
        print("Authors:  %s" % ",".join(author.name for author in authors))

    # get the links to the abs page and pdf for this e-print
    for link in entry.links:
        if link.rel == "alternate":
            print("abs page link: %s" % link.href)
        elif link.title == "pdf":
            print("pdf link: %s" % link.href)

    # The journal reference, comments and primary_category sections live under
    # the arxiv namespace
    journal_ref = getattr(entry, 'arxiv_journal_ref', 'No journal ref found')
    print("Journal reference: %s" % journal_ref)

    comment = getattr(entry, 'arxiv_comment', 'No comment found')
    print("Comments: %s" % comment)

    primary_category = entry.tags[0]["term"]
    print("Primary Category: %s" % primary_category)

    all_categories = [t["term"] for t in entry.tags]
    print("All Categories: %s" % (", ").join(all_categories))

    print("Abstract: %s" % entry.summary)
    print()


Feed title: ArXiv Query: search_query=all:rag&amp;id_list=&amp;start=0&amp;max_results=5
Feed last updated: 2024-04-24T00:00:00-04:00
totalResults for this query: 295
itemsPerPage for this query: 5
startIndex for this query: 0
e-print metadata
arxiv-id: 2404.14043v1
Published: 2024-04-22T09:56:59Z
Title:  LLMs Know What They Need: Leveraging a Missing Information Guided
  Framework to Empower Retrieval-Augmented Generation
Authors:  Keheng Wang,Feiyu Duan,Peiguang Li,Sirui Wang,Xunliang Cai
abs page link: http://arxiv.org/abs/2404.14043v1
pdf link: http://arxiv.org/pdf/2404.14043v1
Journal reference: No journal ref found
Comments: No comment found
Primary Category: cs.CL
All Categories: cs.CL
Abstract: Retrieval-Augmented Generation (RAG) demonstrates great value in alleviating
outdated knowledge or hallucination by supplying LLMs with updated and relevant
knowledge. However, there are still several difficulties for RAG in
understanding complex multi-hop query and retrieving relevant d

In [31]:
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_community.document_transformers import BeautifulSoupTransformer
import json


# Load HTML
links = ["https://habr.com/ru/companies/sberbank/articles/805337/",
         "https://habr.com/ru/feed/"]

loader = AsyncHtmlLoader(links)
html = loader.load()

bs_transformer = BeautifulSoupTransformer()
docs_transformed = bs_transformer.transform_documents(html, tags_to_extract=["span","p"])

result = {}
for i in range(len(links)):
    result[links[i]] = docs_transformed[i].page_content
    
f = open("assets/pages_content.json","w")
f.write(json.dumps(result))
f.close()

Fetching pages: 100%|#########################################################################################################| 2/2 [00:00<00:00,  3.36it/s]


## Инициализация модели
Теперь инициализируем модель GigaChat.

In [25]:
with open("credentials.txt", 'r', encoding='utf-8') as file:
    credentials = file.read()

In [3]:
from langchain.chat_models.gigachat import GigaChat
llm = GigaChat(credentials=credentials, scope="GIGACHAT_API_CORP", verify_ssl_certs=False)

Для проверки спросим у модели вопрос про цвет плаща без какого-либо контекста. Возможно, она и так будет давать ожидаемый ответ...

In [13]:
from langchain.schema import HumanMessage

question = "Какой Loss использует Yolov8?"
llm([HumanMessage(content=question)]).content[0:200]

'YOLOv8 использует несколько видов потерь для обучения модели. Однако, основной вид потери, который используется в большинстве случаев, это Cross-Entropy Loss (CEL). \n\nCross-Entropy Loss (CEL) - это фу'

Видим, что модель не отвечает так, как нам хотелось бы, поэтому применим RAG-подход.

## Подготовка документа

Для работы с документом нам нужно разделить его на части. Для этого используем `TextLoader` для загрузки книги и `RecursiveCharacterTextSplitter`, чтобы разделить текст на приблизительно равные куски в ~1000 символов с перекрытием в ~200 символов. Этот тип сплиттера сам выбирает каким способом следует оптимально разделять документ (по абзацам, по предложениям и т.д.)

In [14]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
)

loader = TextLoader("assets/yolov8_paper.txt")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
documents = text_splitter.split_documents(documents)
print(f"Total documents: {len(documents)}")

Total documents: 46


После нарезки мы получили 91 документ частями книги.

## Создание базы данных эмбеддингов

Эмбеддинг это векторное представление текста, которое может быть использовано для определения смысловой близости текстов. Векторная база данных хранит тексты и соответствующие им эмбеддинги, а также умеет выполнять поиск по ним. Для работы с базой данных мы создаем объект GigaChatEmbeddings и передаем его в базу данных Chroma.

> Обратите внимание, что сервис для вычисления эмбеддингов может тарифицироваться отдельно от стоимости модели GigaChat.

In [15]:
from chromadb.config import Settings
from langchain.vectorstores import Chroma
from langchain_community.embeddings.gigachat import GigaChatEmbeddings

embeddings = GigaChatEmbeddings(
    credentials=credentials, scope="GIGACHAT_API_CORP", verify_ssl_certs=False
)

db = Chroma.from_documents(
    documents,
    embeddings,
    client_settings=Settings(anonymized_telemetry=False),
)

## Поиск по базе данных

Теперь можно обратиться к базе данных и попросить найти документы, которые с наибольшей вероятностью содержат ответ на наш вопрос.

По-умолчанию база данных возвращает 4 наиболее релевантных документа. Этот параметр можно изменить в зависимости от решаемой задачи и типа документов.

Видно, что первый же документ содержит внутри себя часть книги с ответом на наш вопрос.

In [16]:
docs = db.similarity_search(question, k=4)
len(docs)

4

In [17]:
print(f"... {str(docs[0])[620:800]} ...")

... tasets. We also witness YOLOv8 outperforming YOLOv5 for each RF100 category. From Figure 7b we\ncan see that YOLOv8 produces similar or even better results\ncompared to YOLOv5 [16] ...


## QnA цепочка

Теперь мы создадим цепочку QnA, которая специально предназначена для ответов на вопросы по документам. В качестве аргументов есть передается языковая модель и ретривер (база данных).

In [18]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

Наконец можно задать вопрос нашей цепочке и получить правильный ответ!

In [19]:
qa_chain({"query": question})

{'query': 'Какой Loss использует Yolov8?',
 'result': 'YOLOv8 использует функцию потерь, которая состоит из двух частей: локальной потери и потери классификации. Локальная потеря измеряет ошибку между предсказанными областями и истинными областями, в то время как потеря классификации измеряет ошибку между предсказанными классами и истинными классами.'}

Несколько дополнительных вопросов для проверки работоспособности:

In [30]:
qa_chain({"query": "Напиши скрипт для тестирования Yolov8"})

{'query': 'Напиши скрипт для тестирования Yolov8',
 'result': 'Извините, но я не могу написать скрипт для тестирования Yolov8, так как я не знаю, что это такое. Если вы можете предоставить больше информации или контекста, я смогу помочь вам лучше.'}

In [24]:
qa_chain({"query": "Основные особенности Yolov8"})

{'query': 'Основные особенности Yolov8',
 'result': 'YOLOv8 имеет несколько особенностей, которые отличают его от других версий YOLO. Во-первых, он использует новую архитектуру, которая сочетает в себе модули FAN и PAN. Это позволяет ему лучше захватывать особенности на разных масштабах и разрешениях, что важно для точного обнаружения объектов разного размера и формы. Кроме того, YOLOv8 превосходит YOLOv5 по нескольким параметрам, включая более высокую метрику mAP и меньшее количество выбросов при измерении против RF100. Он также превосходит YOLOv5 для каждого RF100 категории.'}